In [1]:
%%html
<style>
table {float:left}
.output {flex-direction: row}
</style>

# Räumliches Zusammenführen von Fahrradunfällen und Tempolimits/RVA

## Initialisierung

### Schritt 1.1: Import von Paketen und Definition von Funktionen

In [2]:
import configparser
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")  # ignore warnings


def deriveSpeedLimit(row):
    default = 50

    if np.isnan(row['index_speed']) & np.isnan(row['index_road']):
        # no intersection with roads or speed limits found
        return None

    if np.isnan(row['index_speed']):
        # road without speed limit -> further check road rank
        if row['rank'] == 3:
            return default
        else:
            return None

    # check whether speed limit was already valid when accident occured
    if row['dat_t'] is not None:
        acc_date_str = f"{row['UJAHR']}-{row['UMONAT']}-15"
        acc_dt = datetime.strptime(acc_date_str, '%Y-%m-%d')
        if acc_dt < row['date']:
            # -> speed limit not yet valid (date)
            return default

    # check day
    if row['tag_t'] is not None:
        if row['UWOCHENTAG'] < row['day_from'] or row['UWOCHENTAG'] > row['day_to']:
            # -> speed limit not yet valid (day)
            return default

    # check time
    if row['zeit_t'] is not None:
        if row['time_from'] < row['time_to']:
            # -> limit during day
            if row['USTUNDE'] < row['time_from'] or row['USTUNDE'] >= row['time_to']:
                # -> speed limit not yet valid (time)
                return default
        else:
            # -> limit during night
            if row['USTUNDE'] < row['time_from'] and row['USTUNDE'] >= row['time_to']:
                # -> speed limit not yet valid (time)
                return default

    # -> valid speed limit determined!
    return row['wert_ves']

### Schritt 1.2: Laden der Konfiguration aus der `config.ini`

In [3]:
# read local config.ini file
rel_path = './../'
config = configparser.ConfigParser()
config.read(rel_path + 'config.ini')

# get from config.ini
dir_output = config['FILE_SETTINGS']['DIR_OUTPUT']
gpkg_src = rel_path + dir_output + config['FILE_SETTINGS']['GPKG_NAME']
gpkg_out = rel_path + dir_output + config['FILE_SETTINGS']['GPKG_NAME_BUF']

### Schritt 1.3: Laden der Unfalldaten

Die Unfall-Rohdaten wurden im vorherigen Workflow-Schritt vorverarbeitet.

Die Daten befinden sich im dafür erstellen GeoPackage `map_data.gpkg` mit folgenden Layern:

| Layer | Daten | Beschreibung |
|:---|:---|:---|
| bike_accidents | Point (19557) | Unfalldaten nach Worfklow-Schritt 1, ohne abgeleitetes Tempolimit/RVA |
| fis_rva | LineString (18641) | Vorprozessierter FIS-Broker Datensatz zu RVA |
| fis_strassenabschnitte | LineString (43346) | Vorprozessierter FIS-Broker Datensatz zu Straßenabschnitten |
| fis_tempolimit | LineString (29424) | Vorprozessierter FIS-Broker Datensatz zu Tempolimits |

In [4]:
df_bike_acc = gpd.read_file(gpkg_src, layer='bike_accidents')
df_roads = gpd.read_file(gpkg_src, layer='fis_strassenabschnitte')
df_speed = gpd.read_file(gpkg_src, layer='fis_tempolimit')
df_rva = gpd.read_file(gpkg_src, layer='fis_rva')

### Schritt 1.4: Spalten umbenennen/löschen

Für dieses Notebook nicht benötigte Attribute werden gelöscht und Spaltennamen aus verschiedenen Datensatzen werden vereinheitlicht bzw. umbenannt.

In [5]:
# delete columns that aren't required for further processing
df_bike_acc.drop(['ULAND', 'UKREIS', 'UGEMEINDE', 'OBJECTID'], axis=1, inplace=True)
df_bike_acc_buf = df_bike_acc.copy()
df_bike_acc_buf.drop(['UKATEGORIE', 'UART', 'UTYP1',
                      'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstig'], axis=1, inplace=True)
df_roads.drop(['strassenna', 'str_bez', 'strassenkl', 'strassen_1', 'strassen_2', 'verkehrsri'], axis=1, inplace=True)
df_rva.drop(['sobj_kz', 'segm_segm', 'rva_typ', 'sorvt_typ', 'b_pflicht'], axis=1, inplace=True)

# rename columns
df_speed.rename(columns={'elem_nr': 'element_nr'}, inplace=True)

***

## Daten vorbereiten

### Schritt 2.1: Initiales Puffern der Linien- bzw. Punktdaten als Vorbereitung für Spatial Joins

- Unfälle: 10m
- Straßenabschnitte und Tempolimits: 15m
- RVA: 5m

*(Hinweis: Die Puffergrößen wurden frei gewählt und sind das Ergebnis vielfältiger Tests. Diese Kombinationen haben insgesamt zu den besten Ergebnissen geführt)*

In [6]:
# create buffer for road/RVA segments and accidents to find a good match
# -> smaller buffer for RVA since each lane is captured separately
df_bike_acc_buf['geometry'] = df_bike_acc_buf['geometry'].buffer(10, resolution=2)
df_roads['geometry'] = df_roads['geometry'].buffer(15, resolution=2, cap_style=2)
df_speed['geometry'] = df_speed['geometry'].buffer(15, resolution=2, cap_style=2)
df_rva['geometry'] = df_rva['geometry'].buffer(5, resolution=2, cap_style=2)

### Schritt 2.2: Gepufferte Geodaten in eigenes GeoPackage `map_data_buffered.gpkg` schreiben, um spätere Analysen zu ermöglichen

In [7]:
df_roads.to_file(gpkg_out, layer='buf_roads', driver='GPKG')
df_speed.to_file(gpkg_out, layer='buf_speed', driver='GPKG')
df_rva.to_file(gpkg_out, layer='buf_rva', driver='GPKG')

### Schritt 2.3: Datenselektion

Im vorherigen Workflow-Schritt wurden u.a. basierend auf dem Straßentyp/-klasse ein Rang für jeden **Straßenabschnitt** ermittelt:

- `0`: nicht relevant für Fahrradunfälle (z.B. Autobahn)
- `1`: Priorität (z.B. Park, Grünanlage, Waldweg, …) &rarr; hier gilt keine Default-Geschwindigkeit
- `2`: (nicht `0`, `1` oder `3`)
- `3`: Priorität (z.B. Bundesstraßen, Gemeindestraßen, …) &rarr; hier gilt eine Default-Geschwindigkeit innerorts von 50km/h

Sowohl der Datensatz für die Straßenabschnitte, als auch der Datensatz für das Tempolimit, enthalten ein Attribute `element_nr`, welches den Straßenabschnitt eindeutig identifiziert. Mittels `merge(..., on="element_nr")` können somit die Ränge aus den Straßenabschnitten zu den dazugehörigen Tempolimits ermittelt werden.

Anschließend können die Datensätze reduziert werden:
- Tempolimits
    - Abschnitte mit Rang `0` löschen &rarr; auf Autobahnen sind keine Fahrradunfälle zu erwarten; falls nicht gelöscht könnten diese jedoch das Ergebnis verfälschen (z.B. wenn Fahrradunfall auf Über-/Unterführung der Autobahn passiert)
    - Datensätze ohne Angabe der zulässigen Höchstgeschwindigkeit löschen &rarr; Geschwindigkeit notwendig für weitere Verarbeitung
- Straßenabschnitte
    - Abschnitte mit Rang `0` löschen &rarr; siehe oben
    - Abschnitte mit Rang `1` löschen &rarr; hierfür gelten keine Default-Geschwindigkeiten, d.h. diese sind für die weitere Verarbeitung nicht relevant

In [8]:
# - speed: ignore road rank '0' (e.g. Autobahn) and records without speed limit
# - roads: ignore road rank '0' (e.g. Autobahn) and '1' (e.g. park, KGA, ...)
df_speed = pd.merge(df_speed, df_roads[['element_nr', 'rank']], how='left', on="element_nr")
df_speed = df_speed.loc[df_speed['rank'] != 0]
df_speed = df_speed.dropna(subset=['wert_ves'])
df_roads = df_roads.loc[df_roads['rank'] > 1]

### Schritt 2.4: Subsetting

Das Tempolimit wird aus der Kombination zweier Datensätze ermittelt:

1. Straßenabschnitte-Datensatz mit einer Default-Geschwindigkeit von 50km/h für Räng `2` und `3`
2. Tempolimit-Datensatz mit zulässiger Höchstgeschwindigkeit als eigenes Attribut

Da die beiden Datensätze nicht disjunkt sind, muss eine Überlagerung vermieden werden. Daher werden insgesamt drei Subsets von Straßenabschnitten gebildet:

1. Abschnitte **ohne** Tempolimit &rarr; Straßenabschnitte-Datensatz abzgl. der Straßenabschnitte, die im Tempolimit-Datensatz über passende `element_nr` vorhanden sind
2. Abschnitte **mit** Tempolimit &rarr; entspricht Tempolimit-Datensatz
3. Abschnitte **mit partiellem** Tempolimit &rarr; Spezialfall, wenn Tempolimit eine zum Straßenabschnitt passende `element_nr` hat, der Abschnitt des Tempolimits jedoch kürzer ist als der gesamte Streckenabschnitt &rarr; wird in einem späteren Schritt ermittelt (`df_acc_rem`)

In [9]:
# reduce roads by matching entries found in speed limits dataset (matching 'element_nr')
df_roads_wo_speed_idx = pd.concat(
    [df_roads['element_nr'], df_speed['element_nr'], df_speed['element_nr']]).drop_duplicates(keep=False)
df_roads_wo_speed = df_roads.loc[df_roads['element_nr'].isin(df_roads_wo_speed_idx.values)]

***

## Spatial Joins: Tempolimits

### Schritt 3.1: Unfälle x Abschnitte ohne Tempolimit &rarr; `df_acc_x_road_wo_speed`

In [10]:
# Spatial join (1): bike accidents for all roads without speed limit
# -> drop duplicates by considering road rank (highest ranked road 'wins')
df_acc_x_road_wo_speed = gpd.sjoin(df_bike_acc_buf, df_roads_wo_speed, how='inner', predicate='intersects',
                                   rsuffix='road')
df_acc_x_road_wo_speed = df_acc_x_road_wo_speed.sort_values(['rank'], ascending=False).drop_duplicates(subset=['GUID'],
                                                                                                       keep='first')

### Schritt 3.2: Unfälle x Abschnitte mit Tempolimit &rarr; `df_acc_x_speed`

In [11]:
# Spatial join (2): bike accidents for all roads with speed limit
df_acc_x_speed = gpd.sjoin(df_bike_acc_buf, df_speed, how='inner', predicate='intersects', rsuffix='speed')

In [12]:
# concatenate all accidents that have been found so far (roads with or without speed limit)
df_concat = pd.concat([df_acc_x_road_wo_speed, df_acc_x_speed], axis=0)

### Schritt 3.3: Unfälle x Abschnitte mit partiellem Tempolimit &rarr; `df_acc_rem_x_road`

In [13]:
# specialty: speed limits sometimes don't cover the entire length of the road, although they have matching IDs
# -> some roads have been ignored in spatial join (1) because it was assumed intersect was found in speed limit DF
# -> extract those missing entries and spatial join (3) again with roads
df_acc_rem_guid = pd.concat([df_bike_acc_buf['GUID'], df_concat['GUID']]).drop_duplicates(keep=False)
df_acc_rem = df_bike_acc_buf.loc[df_bike_acc_buf['GUID'].isin(df_acc_rem_guid.values)]
df_acc_rem_x_road = gpd.sjoin(df_acc_rem, df_roads, how='left', predicate='intersects', rsuffix='road')

In [14]:
# append/concat remaining accidents
df_concat = pd.concat([df_concat, df_acc_rem_x_road], axis=0)

**&rarr; Zwischenergebnis**: jedem Unfall wurden die jeweils relevanten Streckenabschnitte aus den beiden Datensätzen (Straßenabschnitte bzw. Tempolimits) zugeordet &rarr; `df_concat`

*(Hinweis: mit Außnahme der Unfälle, die nicht auf kartierten Straßenabschnitten passiert sind)*

***

## Tempolimit ableiten

### Schritt 4.1: Tempolimit aus Streckenabschnitt ableiten &rarr; Funktion `deriveSpeedLimit()` (vgl. Schritt 1.1)

Nachdem jedem Unfall via Spatial Join mind. ein Streckenabschnitt zugeordnet wurde, muss aus dem jeweiligen Streckenabschnitt das zum Unfallzeitpunkt gültige Tempolimit abgeleitet werden. Dabei wird berücksichtigt:

- Typ des Streckenabschnitts
- Datum der Einführung des Tempolimits
- Gültigkeit des Tempolimits (Einschränkung nach Wochentag und/oder Uhrzeit)

In [15]:
df_concat['speed_der'] = df_concat.apply(deriveSpeedLimit, axis=1).astype('Int64')

### Schritt 4.2: Duplikate entfernen

Zu diesem Zeitpunkt kann es sein, dass ein Unfall mehreren Streckenabschnitten zugeordnet ist (z.B. aufgrund Kreuzung). In diesem Fall wird der Streckenabschnitt mit der **höchsten** Geschwindigkeit beibehalten, alle anderen werden gelöscht &rarr; `df_concat_u`

In [16]:
# since accidents at intersections have multiple road matches
# -> drop duplicates (road with the highest speed limit 'wins')
# -> remove unnecessary columns
# -> count nan values (no speed limit derived)
df_concat_u = df_concat.sort_values(['speed_der'], ascending=False).drop_duplicates(subset=['GUID'], keep='first')
df_concat_u.drop(['index_road', 'index_speed', 'zeit_t', 'tag_t', 'dat_t', 'laenge'], axis=1, inplace=True)
nan_count_speed = df_concat_u['speed_der'].isna().sum()

In [17]:
print(f'Anzahl Unfälle, für die kein Tempolimit ermittelt werden konnte: {nan_count_speed}')

Anzahl Unfälle, für die kein Tempolimit ermittelt werden konnte: 23


### Schritt 4.3: Unfalldaten inkl. Tempolimit werden GeoPackage hinzugefügt

In [18]:
# - 10m buffered accidents (incl. speed limit)
df_concat_u.to_file(gpkg_out, layer='buf_acc_speed', driver='GPKG')

***

## Spatial Joins: RVA

### Schritt 5.1: Puffer anpassen

Da RVA im Gegensatz zu Straßenabschnitten und Tempolimits richtungsabhängig sind, wird der Puffer von Unfällen um -5m auf insgesamt 5m angepasst. Somit ergibt sich:

- Unfälle: 5m
- RVA: 5m

*(Hinweis: Die Puffergrößen wurden frei gewählt und sind das Ergebnis vielfältiger Tests. Diese Kombination hat insgesamt zu den besten Ergebnissen geführt)*

In [19]:
# copy dataframe and reduce buffer of accidents
df_acc = df_concat_u.copy()
df_acc['geometry'] = df_acc['geometry'].buffer(-5, resolution=2, cap_style=2)

### Schritt 5.2: Unfälle x RVA &rarr; `df_acc_x_rva`

Da in diesem Fall die Daten nicht aus mehreren Datensätzen kombiniert werden müssen, reicht ein einfacher Spatial Join. 

Auch hier kann es dazu kommen, dass ein Unfall mehreren RVA zugeordnet wird. In diesem Fall wird die RVA mit dem **niedrigesten** Rang beibehalten, alle anderen werden gelöscht.

In [20]:
# spatial join with RVA
# -> drop duplicates (RVA with lowest rank/safety 'wins')
# -> remove unnecessary columns
# -> replace nan values to keep accidents where no RVA could be found
df_acc_x_rva = gpd.sjoin(df_acc, df_rva, how='left', predicate='intersects', lsuffix='road', rsuffix='rva')
df_acc_x_rva = df_acc_x_rva.sort_values(['rank_rva'], ascending=False).drop_duplicates(subset=['GUID'], keep='last')
df_acc_x_rva.drop(['index_rva'], axis=1, inplace=True)
df_acc_x_rva['rank_rva'].fillna(0, inplace=True)
df_acc_x_rva['rank_rva'] = df_acc_x_rva['rank_rva'].astype('Int64')

### Schritt 5.3: Unfalldaten inkl. RVA werden GeoPackage hinzugefügt

In [21]:
# - 5m buffered accidents (incl. RVA)
df_acc_x_rva.to_file(gpkg_out, layer='buf_acc_rva', driver='GPKG')

***

## Punktdaten zu den Unfällen erweitern &rarr; `df_bike_acc_out`

### Schritt 6.1: Initialen Unfall-Datensatz um ermitteltes Tempolimit und RVA erweitern (`merge`)

In [22]:
df_bike_acc_out = pd.merge(df_bike_acc, df_concat_u[['GUID', 'speed_der']])
df_bike_acc_out = pd.merge(df_bike_acc_out, df_acc_x_rva[['GUID', 'rank_rva']])

### Schritt 6.2: Unfälle kennzeichnen, bei denen die Geschwindigkeit relevant ist

Dazu zählen Fahrradunfälle, wo außerdem folgende Verkehrsteilnehmer beteiligt waren:

- Unfälle mit PKW
- Unfälle mit Krad
- Unfälle mit Güterkraftfahrzeug (z.B. LKW)
- Unfälle mit Sonstigen (z.B. Bus, Tram, ...)

Bei anderen Verkehrsteilnehmern, wie z.B. Unfall mit Fußgängern, ist das Tempolimit an der Unfallstelle für die weitere Auswertung nicht relevant.

In [23]:
df_bike_acc_out['speed_rel'] = np.where((df_bike_acc_out['IstPKW'] == 1) |
                                        (df_bike_acc_out['IstKrad'] == 1) |
                                        (df_bike_acc_out['IstGkfz'] == 1) |
                                        (df_bike_acc_out['IstSonstig'] == 1),
                                        True, False)

### Schritt 6.3:  Unfalldaten inkl. Tempolimit und RVA werden GeoPackage `map_data.gpkg` hinzugefügt

In [24]:
df_bike_acc_out.to_file(gpkg_src, layer='bike_accidents_ext', driver='GPKG')